In [ ]:
import pandas  as pd
import glob,os
import matplotlib.pyplot as plt
import matplotlib.style as style
import warnings
from datetime import datetime
import numpy as np
import seaborn as sns

In [ ]:
#配置
warnings.filterwarnings("ignore")
pd.set_option("display.float_format","{:.4f}".format)
style.use("ggplot")
plt.rcParams["font.sans-serif"] = ["SimHei"]
plt.rcParams["axes.unicode_minus"] = False
plt.rcParams["figure.figsize"] = (20,10)
plt.rcParams["figure.dpi"] = 200

In [ ]:
os.chdir("/home/jovyan/work/data/竞争数据/商品销售数据/")


In [ ]:
files = glob.glob("./*.xlsx")
dfs = []
for f in files:
    t = f[2:4]
    df = pd.read_excel(f)
    #如果t == "拜耳"  使用对象  30天销售额
    if t=="拜耳":
        df.rename(columns={"销售额":"30天销售额","使用对象":"适用对象"},inplace=True)
    dfs.append(df)

In [ ]:
data = pd.concat(dfs,ignore_index=True)

In [ ]:
data.drop(columns=["主图链接","商品链接","物理形态","型号","净含量","序号"],inplace=True)

# 1.三家共同点在哪里

In [ ]:
d = data.query("类目 =='灭鼠/杀虫剂'").groupby(by=["店铺名称","适用对象"])["30天销售额"].sum().unstack(level=0).fillna(0)

In [ ]:
plt.figure(figsize=(10,8))
sns.heatmap(data=d,annot=True,fmt=".2f",linewidths=1,cmap="Blues")

# 2.各各品类产品交易指数对比

In [ ]:
os.chdir("../商品交易数据/")
files = glob.glob("./*.xlsx")

In [ ]:
dfs = []
for f in files:
    t = f[2:4]
    df = pd.read_excel(f)
    df["品牌"] = t
    if t == "拜耳":
        df.rename(columns={"时间":"日期"},inplace=True)
    dfs.append(df)
    

In [ ]:
data = pd.concat(dfs,ignore_index=True)

In [ ]:
data["品类"] = np.nan

In [ ]:
for x in d.index[2:]:
    data.loc[data.商品.str.contains(x),"品类"] = x 

In [ ]:
for x in d.index[2:]:
    display(x,data.query("品类 == @x").groupby("品牌").交易指数.sum())

# 3.市场上商品结构

- 相对市场增长率
- 相对市场占有率

In [ ]:
#商品中有重复值
data = pd.merge(data.query("品类 == '蟑螂'"),data.groupby("商品").交易指数.sum().reset_index(0).rename(columns={"交易指数":"交易指数和"}),left_on="商品",right_on="商品")

In [ ]:
#1代表当商品只有一家在售卖
data["同种类商品售卖比例"] = data["交易指数"].div(data["交易指数和"])

In [ ]:
#调和均值
data["交易增长幅度调和值"] = data["交易增长幅度"].mul(data["同种类商品售卖比例"])

In [ ]:
df = data.groupby("商品")\
.agg({"交易指数":"sum","交易增长幅度调和值":"sum","商品":"count","品牌":"max"})\
.rename(columns={"交易指数":"相对市场占有率","交易增长幅度调和值":"相对市场增长率","商品":"产品数量"})

In [ ]:
display(df.describe(percentiles=[0.8,0.85,0.9,0.95,0.99,1]))  #描述统计

# 找到变化率不太快的位置  , 让大值 = 0.85位置的数值

def quantile(x,q=0.85):
    qu = x.quantile(q)  # 找到0.85百分的值
    return x.mask(x > qu, qu)   # mysql 当中 if(cond,True,False)  mask有循环的功效
    
def action(df):
    df = df.copy() #希望原先的数据不被修改
    df["相对市场占有率"] = quantile(df["相对市场占有率"])
    df["相对市场增长率"] = quantile(df["相对市场增长率"])
    return df

In [ ]:
result = action(df)

In [ ]:
def BCGPlot(df,is_mean=True,xq=0.5,yq=0.5): #默认使用均值进行分割
    axes = sns.scatterplot(df,x="相对市场占有率",y="相对市场增长率",size="产品数量",hue="品牌")
    if is_mean:
        plt.axvline(df.相对市场占有率.mean(),c="k",ls=":",lw=1)
        plt.axhline(df.相对市场增长率.mean(),c="k",ls=":",lw=1)
    else:
        plt.axvline(df.相对市场占有率.quantile(xq),c="k",ls=":",lw=1)
        plt.axhline(df.相对市场增长率.quantile(yq),c="k",ls=":",lw=1)
    
    plt.text(250000,.8,"明星类",fontsize=20,color="y")
    plt.text(250000,-.5,"奶牛类",fontsize=20,color="r")
    plt.text(100000,.8,"山猫类",fontsize=20,color="b")
    plt.text(100000,-.5,"瘦狗类",fontsize=20,color="k")

In [ ]:
BCGPlot(result,is_mean=True)

In [ ]:
BCGPlot(result,is_mean=False)

# 给数据打上标签

In [ ]:
def target(df,is_mean=True,xq=0.5,yq=0.5):
    相对市场占有率均值 = df["相对市场占有率"].mean()
    相对市场增长率均值 = df["相对市场增长率"].mean()

    df.loc[(df["相对市场占有率"] >= 相对市场占有率均值) & (df["相对市场增长率"] >= 相对市场增长率均值),"产品结构"] = "明星类"
    df.loc[(df["相对市场占有率"] >= 相对市场占有率均值) & (df["相对市场增长率"] < 相对市场增长率均值),"产品结构"] = "金牛类"
    df.loc[(df["相对市场占有率"] < 相对市场占有率均值) & (df["相对市场增长率"] >= 相对市场增长率均值),"产品结构"] = "山猫类"
    df.loc[(df["相对市场占有率"] < 相对市场占有率均值) & (df["相对市场增长率"] < 相对市场增长率均值),"产品结构"] = "瘦狗类"
    
    df.loc[(df["相对市场占有率"] >= 相对市场占有率均值) & (df["相对市场增长率"] >= 相对市场增长率均值),"序号"] = 2
    df.loc[(df["相对市场占有率"] >= 相对市场占有率均值) & (df["相对市场增长率"] < 相对市场增长率均值),"序号"] = 1
    df.loc[(df["相对市场占有率"] < 相对市场占有率均值) & (df["相对市场增长率"] >= 相对市场增长率均值),"序号"] = 3
    df.loc[(df["相对市场占有率"] < 相对市场占有率均值) & (df["相对市场增长率"] < 相对市场增长率均值),"序号"] = 4
    
    return df.sort_values(by="序号")

In [ ]:
result = target(result)

In [ ]:
d = pd.pivot_table(data=result,index="产品结构",columns="品牌",values="产品数量",aggfunc="count").fillna(0)

In [ ]:
sns.heatmap(data=d.loc[result.产品结构.unique()],annot=True,cmap="autumn",linewidths=1)

# 流量

- ROI : 投资回报比
- 不同的渠道, 带来的用户质量(客单价  退货率  转换程付费率)  微信视频
- 投放广告的周期 ; 50W   7点 开投 - 9店  10点 
- 不同的人群 
- 不同的广告投放方式


- 付费流量
- "免费"流量
    - 淘宝客 : 自己生成链接 , 由淘宝客 推 链接出去 , 别人点击了链接 并且 购买了,则有提成 
    - SEO优化 : 关键字(关键字热度) 

In [ ]:
os.chdir("../流量渠道数据/")

In [ ]:
files = glob.glob("./*.xlsx")
dfs = []
for f in files:
    t = f[2:4]
    df = pd.read_excel(f)
    df["品牌"] = t
    if t=="拜耳" : 
        df.drop(columns=["交易指数.1"],inplace=True)
    dfs.append(df)

In [ ]:
data = pd.concat(dfs,ignore_index=True)

In [ ]:
df = data.groupby(by=["流量来源","品牌"]).交易指数.sum().unstack(level=-1).replace({0:np.nan}).dropna(how="all")

In [ ]:
df1 = df.dropna(how="any")

In [ ]:
#后台 数据中知道 当前的用户 从什么渠道 点击链接  进入到 淘宝 付费
df1.plot(kind="bar")

# 用户评价

In [ ]:
os.chdir("../评论舆情数据/")

In [ ]:
files = glob.glob("./*.xlsx")
dfs = []
for f in files:
    t = f[2:4]
    df = pd.read_excel(f)
    df["品牌"] = t
    
    dfs.append(df)
    

In [ ]:
data = pd.concat(dfs,ignore_index=True)

In [ ]:
data = data[["品牌","评论"]]

- 注意舆情中的 表情 最好能够去除 \u00000 - \ufffff
- 中文分词 
- 对有用的词汇进行计数
- 单个字都是 没有作用

## 剔除无用数据

In [ ]:
import re

In [ ]:
data["评论"]=data.评论.map(lambda x : re.sub(r"[^\u000000-\uffffff]+","",x))

In [ ]:
data = data.loc[data.评论 != "此用户没有填写评论"]

## 中文分词

In [ ]:
!pip install jieba 

In [ ]:
!pip install wordcloud

In [ ]:
import jieba  #结巴
from wordcloud import WordCloud

In [ ]:
jieba.__version__

In [ ]:
jieba.load_userdict("./userdict.txt")

In [ ]:
#停用词
stopwords = pd.read_table("../../百度停用词表.txt",header=None)[0]

In [ ]:
def fo(x):
    return [w for w in jieba.lcut(x) if len(w)>1]

mask = plt.imread("../../leaf.jpg") 

def wcplot(d):
    
    #去除停用词  我认为这些词汇 可能没有用处
    w = d.explode()
    
    w = w.loc[~w.isin(stopwords)]  #去除停用词
    
    c = w.value_counts()  #词计数
    
    #当前只是图片的配置项 返回词云画布
    wc = WordCloud(font_path="../../SimHei.ttf",mask=mask,width=1000,height=800,min_font_size=10,max_font_size=100,background_color="white")
    
    wc.fit_words(c)
    
    plt.imshow(wc)
    
    plt.axis("off")

In [ ]:
d = data.query("品牌 == '拜耳'").评论.map(fo)

wcplot(d)

In [ ]:
d = data.query("品牌 == '科林'").评论.map(fo)

wcplot(d)

In [ ]:
d = data.query("品牌 == '安速'").评论.map(fo)

wcplot(d)

# 情感分析

In [ ]:
def fo(x):
    s = pd.Series([w for w in jieba.lcut(x) if len(w)>1])
    return " ".join(s.loc[~s.isin(stopwords)])
    

textdata = data.评论.map(fo)

In [ ]:
textdata.iloc[6]

In [ ]:
a = data.loc[data.评论.str.contains("垃圾|完全没有效果|小强依然活跃")]
a["评价"] = 0

In [ ]:
b= data.loc[data.评论.str.contains("必须好评|效果很好|强烈推荐")]
b["评价"] = 1

In [ ]:
ab = pd.concat([a,b])
ab

## 贝叶斯分析 

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer #把文字转换 成数字  #词向量

In [ ]:
tfidf = TfidfVectorizer()

feature = tfidf.fit_transform(ab.评论) #179行数据  682个词汇

pred_feature = tfidf.transform(textdata) #更加上面682个词汇 把 当前的数据 也定义成  词向量

target = ab.评价

In [ ]:
from sklearn.naive_bayes import BernoulliNB  #二项分布  添加越策标签

In [ ]:
model = BernoulliNB().fit(feature,target)

In [ ]:
pred_target = model.predict(pred_feature)

In [ ]:
data["评价"] = pred_target

In [ ]:
data.评价